In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd


chrome_options = Options()

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)


urls = {
    "Điện thoại": "https://www.lazada.vn/catalog/?q=điện%20thoại",
    "Quần áo": "https://www.lazada.vn/catalog/?q=quần%20áo",
    "Tủ lạnh": "https://www.lazada.vn/catalog/?q=tủ%20lạnh",
    "Tivi": "https://www.lazada.vn/catalog/?q=ti%20vi"
}


def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(5):  # scroll 5 lần (có thể tăng nếu muốn nhiều hơn)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


def crawl_lazada(url, category, max_items=30):
    driver.get(url)
    time.sleep(4)

    scroll_page()

    data = []
    products = driver.find_elements(By.CSS_SELECTOR, "div.RfADt")
    print(f"[{category}] tìm thấy {len(products)} sản phẩm")

    for p in products[:max_items]:
        try:
            name = p.text.strip()
        except:
            name = "N/A"

        try:
            price = p.find_element(By.XPATH, ".//following::span[@class='ooOxS'][1]").text
        except:
            price = "N/A"

        try:
            sold_count = p.find_element(By.XPATH, ".//following::span[contains(text(),'Đã bán')]").text
        except:
            sold_count = "N/A"

        data.append({
            "Category": category,
            "Name": name,
            "Price": price,
            "SoldCount": sold_count
        })

    return data


all_data = []
for category, url in urls.items():
    print(f"Đang crawl {category} ...")
    results = crawl_lazada(url, category, max_items=30)
    all_data.extend(results)

df = pd.DataFrame(all_data)
df.to_excel("lazada_products.xlsx", index=False)
driver.quit()
print(" Crawl xong! Dữ liệu lưu vào lazada_products.xlsx")
